In [1]:
import importlib
import dbops
import cv2
import time
import numpy as np
from tqdm import tqdm
from PIL import Image

import torch
from torchvision import transforms
from torchvision.transforms import functional as F

from facelib import FaceRecognizer, get_config, FaceDetector

import glob
import math

/scratch/u1319464/miniforge3/envs/ds4150-faces/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
!nvidia-smi

Mon Apr 14 12:25:49 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.230.02             Driver Version: 535.230.02   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1070        Off | 00000000:01:00.0 Off |                  N/A |
| 27%   32C    P8               6W / 151W |    982MiB /  8192MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
importlib.reload(dbops)  # Reload the module from source

<module 'dbops' from '/scratch/u1319464/Data-Mining-Spring-2025-Project/dbops.py'>

In [4]:
facelib_conf = get_config()
face_regonizer = FaceRecognizer(facelib_conf)


In [5]:
facelib_conf

{'data_path': PosixPath('/scratch/u1319464/miniforge3/envs/ds4150-faces/lib/python3.8/site-packages/facelib/InsightFace/models/data'),
 'work_path': PosixPath('weights'),
 'model_path': PosixPath('weights/models'),
 'log_path': PosixPath('weights/log'),
 'save_path': PosixPath('weights'),
 'input_size': [112, 112],
 'embedding_size': 512,
 'use_mobilfacenet': True,
 'net_depth': 50,
 'drop_ratio': 0.6,
 'net_mode': 'ir_se',
 'device': device(type='cuda', index=0),
 'data_mode': 'emore',
 'vgg_folder': PosixPath('/scratch/u1319464/miniforge3/envs/ds4150-faces/lib/python3.8/site-packages/facelib/InsightFace/models/data/faces_vgg_112x112'),
 'ms1m_folder': PosixPath('/scratch/u1319464/miniforge3/envs/ds4150-faces/lib/python3.8/site-packages/facelib/InsightFace/models/data/faces_ms1m_112x112'),
 'emore_folder': PosixPath('/scratch/u1319464/miniforge3/envs/ds4150-faces/lib/python3.8/site-packages/facelib/InsightFace/models/data/faces_emore'),
 'batch_size': 100,
 'facebank_path': PosixPath(

In [6]:

EMB_METHOD = 'FaceLib.FaceRecognizer(FaceLib.get_config())'

transform = transforms.Compose([
    transforms.Resize((112, 112)),
    # transforms.ToTensor()
])

def do_embedding(faces):
    if len(faces) > 0:
        with torch.no_grad():
            # from [0,255] RGB to [0,1] BGR <-- important
            faces = faces[:,[2,0,1]].float().div(255).to(face_regonizer.device)
            embeddings = face_regonizer.feature_extractor(faces)
            return embeddings
    else:
        raise Exception('no faces to parse')


In [10]:

for movie_ref in dbops.Movie.iterate_all_movies():
    imdb_id = movie_ref.get_imdb_id()
    title = movie_ref.get_title()
    file_path = movie_ref.get_file_path()

    for frame_ref in tqdm(movie_ref.iterate_frames(), desc=movie_ref.get_title()):
        
        for face_ref in frame_ref.iterate_faces():

            emb_ref = dbops.FaceIdentity.add_face(face_ref.imdb_id, face_ref.frame_index, face_ref.face_index, EMB_METHOD)

            try:
            
                aligned_face_img = face_ref.get_face_image_aligned().resize([112, 112])
    
                embeddings = do_embedding(F.pil_to_tensor(aligned_face_img).permute(1, 2, 0).unsqueeze(0))
    
                # print(embeddings.cpu().numpy(), classifications, confidences)
                
                emb_ref.set_embedding(embeddings[0].cpu().numpy())
    
                # print(emb_ref.get_embedding())

            except Exception as e:
                print(e)


The Invisible Man (1933): 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 2263/2263 [00:39<00:00, 56.70it/s]


In [9]:
F.pil_to_tensor(aligned_face_img).permute(1, 2, 0).unsqueeze(0).shape

torch.Size([1, 112, 112, 3])

In [11]:
embeddings.shape

torch.Size([1, 512])